# Implementación en arquitectura **CPU**

In [3]:
import numpy as np
import os
import gzip

## Función para cargar el set de datos de validación junto con la categoria 
## a la que pertenece cada una de las imagenes, es decir, el homologo en modelos a la variable
## respuesta y.

def load_mnist(ruta, tipo='train'):

    ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
    ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
    with gzip.open(ruta_categorias, 'rb') as rut_cat:
        etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

    with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
        imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

    return imagenes, etiquetas


In [4]:
### Carga de datos

## Con este código se carga la libreria Drive de google colab, que permite realizar la lectura de datos
## desde google drive, es decir establece la conexión entre Drive & Colab
from google.colab import drive

## Con este código se monta el drive en el servidor remoto de Google Colab
drive.mount('/content/gdrive')
## Luego se define la ruta completa donde se encuentran almacenados los datos 
ruta = 'gdrive/My Drive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_mnist(ruta, tipo='train')
X_test, Y_test = load_mnist(ruta, tipo='test')

## Se ejecua esta función, el programa accede a la ruta de drive especificada y por seguridad
## nos pide un código de autorización para poder acceder a los datos almacenados en nuestra carpeta, ya con esto
## se llevan los datos desde nuestra carpea a los servidores de google colab. 


Mounted at /content/gdrive


### Ajusta el tamaño de los datos

Antes de crear el modelo se debe hacer dos ajuses en el set de datos:

1.   Se debe garantizar que el set de entrenamiento contenga un numero de registros que sea exactamente un múltiplo de 128, es necesario por que en el caso de la TPU la arquitectura requiere precisamente que durante el entrenamiento presentemos bloques de datos que sean múltiplos de 128, esto no es necesario para la CPU y GPU, pero acá se deja definido de una vez para poder comparar mas adelante los resultados con las otras dos arquitecturas, la siguiente es una forma de calcular el número mas cercano a 60000 y que a su ves es múltiplo de 128:

$$\frac{60000}{128}*128 = 59904$$

2.   Ajuste de forma equivalente con el set de validación

$$\frac{10000}{128}*128 = 9984$$





In [6]:
## Se reajusta el set de entrenamiento a tamaño 59904 y el set de validación a 9984
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]
## Se utiliza rechape de la libreria Numpy para reajustar el tamaño de los set de datos, y asi garantizar
## que cada dato es una imagen en escala de grises de 28 x 28 pixeles, que sera la entrada a la red convolucional 
## que queremos entrenar.
X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

### Importar TensorFlow 2 (Incluye Keras)

In [7]:
# Importar la libreria TF 2, en este caso teniendo en cuenta que la plataforma es Google Colab,
# se debe  especificar que se quiere importar la versión 2 
%tensorflow_version 2.x   # Para garantizar que la versión 2.x sea importada
import tensorflow as tf
print('Versión de TensorFlow: ' + tf.__version__)


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x   # Para garantizar que la versión 2.x sea importada`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.
Versión de TensorFlow: 2.4.1


Para implementar el modelo se utiliza la libreria Keras, que viene incorporada en la versión 2 de TF.

In [9]:

tf.random.set_seed(200)
# Se crea el contenedor del modelo utilizando el módulo "Sequential", y luego 
# se agregan progresivamente las 3 capas convolucionales, las tres capas son prácticamente
# identicas.
model = tf.keras.models.Sequential()

# Primero se hace una normalización de los datos para facilitar la convergencia del
# entrenamieno, eso se hace con la función "BatchNormalization"
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))

# Se agregan los filtros convolucionales con la función "Conv2D", que contiene
# los siguientes parámetros: "Conv2D(64, (5, 5), padding='same', activation='elu')"
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))

# Luego se agrega una capa "MaxPooling2D" que contiene los siguientes parámetros:
# "MaxPooling2D(pool_size=(2, 2), strides=(2,2))"
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# Y finalmente una capa "Dropout" para reducir el "Overfitting" del modelo
model.add(tf.keras.layers.Dropout(0.25))

# La única diferencia entre las 3 capas radica en el número de filtros convolucionales
# utilizados (64, 128 y 256)

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

# Después de la tercera capa convolucional, se aplanan los datos usando el módulo "Flatten"
# y se agrega una red neuronal con 256 neuronas ("Dense") que posteriormene se conecta a la salida
# la cual contendrá 10 neuronas y una función de activación ("softmax") para clasificar cada una de las
# imagenes que ingresan
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))

# Se imprimen los resultados del modelo, la implementación en Google Colab es exactamene
# igual que como si se esuviese ejecutando localmente en nuestro pc
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 128)        

In [10]:
# Antes del entrenamiento, se compila el modelo, es decir que se define el optimizador a utilizar ("adam"),
# la función de error ("crossentropy") y la métrica de desempeño ("Precisión")
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

### Entrenamiento con CPU

In [ ]:
# Se procede con el entrenamiento de la red, como es un modelo complejo que cuenta con 1'619.470 parámetros
# y se esta ejecutando en la arquitectura de CPU, el proceso es demasiado lento, en total tarda aproximadamene 1539 segundos.

# Por este particular solo se realizan 2 iteraciones de entrenamiento (epochs)
import timeit

# Los parámetros del modelo son:
# Set de entrenamiento para X, Set de entrenamiento para Y, set´s de validación, numero de registros que contiene
# el lote de registros y número de iteraciones "epochs"
def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

Epoch 1/2
468/468 [==============================] - 771s 2s/step - loss: 1.0169 - accuracy: 0.7078 - val_loss: 0.4264 - val_accuracy: 0.8551
Epoch 2/2
468/468 [==============================] - 768s 2s/step - loss: 0.4105 - accuracy: 0.8562 - val_loss: 0.3127 - val_accuracy: 0.8922


In [ ]:
print('Tiempo de entrenamiento: ' + str(cpu_time) + ' segundos')

Tiempo de entrenamiento: 1539.2389175629999 segundos
